In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/yiweizhao/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


import data from excel, save as "df"

In [2]:
import pandas as pd
import numpy as np
df = pd.read_excel("1860-1930 database for family tracking Adam manual AFAM CANADA current.xlsx")
df.shape

(50858, 48)

We believe "firstname", "lastname", "age", "place" and "sex" are good features to predict "match".
Here is the small dataset named "small_data" only with above columns.

In [3]:
small_data = df[['First.Name','Last.Name','Age','Place','Sex']]

In [62]:
# drop rows with at least one NaN
small_data_drop_na=small_data.dropna()
# drop rows with question mark
small_data_drop_na = small_data_drop_na[(small_data_drop_na != '?').all(axis=1)]
small_data_drop_na.shape

(35262, 5)

In [95]:
print(type(small_data_drop_na.iloc[35261]['Age']))
print(type(small_data_drop_na.iloc[1]['Age']))
small_data_clean=small_data_drop_na[small_data_drop_na['Age'].apply(lambda x: type(x) != str)]
small_data_clean.shape

<class 'str'>
<class 'int'>


(35236, 5)

In [96]:
small_data_clean = small_data_clean[small_data_clean.Age < 130]
small_data_clean.shape

(35229, 5)

In [97]:
small_data_clean = small_data_clean.iloc[:3000,:] #only test the first 3000 rows
import time
start_time = time.time()
length=len(small_data_clean)
res = [ [ 0 for i in range(length) ] for j in range(length) ] 
for i in range(length):
    for j in range(i+1,length):
        score = 0
        for z in range(5):        
            if(z!=2): #fuzz cannot handle numerical data
                try: 
                    score += fuzz.ratio(small_data_clean.iloc[i][z],small_data_clean.iloc[j][z])
                except:
                    pass
            if(z==2): #age
                score += 100-abs(small_data_clean.iloc[i][2]-small_data_clean.iloc[j][2])
        res[i][j]= score
print("--- %s seconds ---" % (time.time() - start_time))

--- 4967.821900844574 seconds ---


Since the real the score matrix is symmetric with diagonal max score. We only keep the upper side of this score matrix

In [145]:
df = pd.DataFrame(res) 
df 

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,199,172,99,102,210,160,240,95,205,...,96,79,55,55,178,68,160,197,158,72
1,0,0,100,159,158,193,77,206,153,239,...,97,119,20,101,196,55,217,151,131,113
2,0,0,0,201,204,218,238,193,197,96,...,205,174,206,161,86,175,52,140,87,162
3,0,0,0,0,332,184,240,102,218,132,...,237,227,150,292,121,148,101,63,89,230
4,0,0,0,0,0,196,203,105,201,138,...,190,232,136,279,169,134,120,48,50,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,120,192,170,197
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,217,198,163
2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,298,98
2998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,102


In [146]:
index = 0
max_values = [ [ 0 for i in range(3) ] for j in range(50) ] 
for i in range(len(df)):
    for j in range(len(df)):
        if df[i][j] > 480:
            max_values[index][0] = i
            max_values[index][1] = j
            max_values[index][2] = df[i][j]
            index = index + 1
max_values

[[170, 150, 489],
 [274, 78, 490],
 [365, 364, 500],
 [418, 163, 495],
 [433, 393, 493],
 [528, 393, 481],
 [528, 433, 488],
 [546, 423, 487],
 [563, 523, 491],
 [577, 576, 500],
 [666, 563, 482],
 [717, 373, 493],
 [943, 501, 496],
 [1086, 1085, 500],
 [1099, 1098, 500],
 [1240, 921, 493],
 [1251, 1250, 490],
 [1426, 995, 494],
 [1483, 1482, 500],
 [1545, 1544, 497],
 [1582, 1581, 500],
 [1693, 687, 490],
 [1751, 1441, 492],
 [1899, 1239, 495],
 [1915, 1682, 489],
 [1919, 1918, 500],
 [2003, 828, 486],
 [2032, 2031, 490],
 [2090, 2089, 500],
 [2174, 2007, 489],
 [2183, 2181, 491],
 [2231, 925, 490],
 [2286, 1973, 492],
 [2288, 2287, 481],
 [2302, 2301, 500],
 [2314, 2184, 484],
 [2315, 2184, 491],
 [2325, 2324, 481],
 [2386, 2080, 482],
 [2412, 2411, 483],
 [2505, 2002, 500],
 [2752, 2751, 500],
 [2798, 2797, 500],
 [2801, 2800, 500],
 [2844, 2375, 498],
 [2846, 1907, 485],
 [2928, 2927, 489],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [148]:
max_values = pd.DataFrame(max_values) 
max_values.columns = ['row1', 'row2', 'value']

In [202]:
matched_pair = [ [ 0 for j in range(13) ] for i in range(50) ] 
for i in range(50):
    matched_pair[i][0]=small_data_clean.iloc[max_values['row1'][i]]['First.Name']
    matched_pair[i][1]=small_data_clean.iloc[max_values['row1'][i]]['Last.Name']
    matched_pair[i][2]=small_data_clean.iloc[max_values['row1'][i]]['Age']
    matched_pair[i][3]=small_data_clean.iloc[max_values['row1'][i]]['Place']
    matched_pair[i][4]=small_data_clean.iloc[max_values['row1'][i]]['Sex']
    matched_pair[i][5]=small_data_clean.iloc[max_values['row1'][i]].name
    matched_pair[i][6]=small_data_clean.iloc[max_values['row2'][i]]['First.Name']
    matched_pair[i][7]=small_data_clean.iloc[max_values['row2'][i]]['Last.Name']
    matched_pair[i][8]=small_data_clean.iloc[max_values['row2'][i]]['Age']
    matched_pair[i][9]=small_data_clean.iloc[max_values['row2'][i]]['Place']
    matched_pair[i][10]=small_data_clean.iloc[max_values['row2'][i]]['Sex']
    matched_pair[i][11]=small_data_clean.iloc[max_values['row2'][i]].name
    matched_pair[i][12]=max_values['value'][i]

In [203]:
matched_pair = pd.DataFrame(matched_pair)
matched_pair.columns = ['First.Name1', 'Last.Name1','Age1', 'Place1', 'Sex1', 'Index1',
                        'First.Name2', 'Last.Name2','Age2', 'Place2', 'Sex2', 'Index2','value']
matched_pair

,First.Name1,Last.Name1,Age1,Place1,Sex1,Index1,First.Name2,Last.Name2,Age2,Place2,Sex2,Index2,value
0,Coleman,Freeman,64,Windsor,M,324,Coleman,Freeman,75,Windsor,M,292,489
1,John,Murphey,50,Bangor,M,475,John,Murphey,60,Bangor,M,156,490
2,R,Emery,58,St. George's Ward,M,628,R,Emery,58,St. George's Ward,M,627,500
3,James,Wells,60,Windsor,M,709,James,Wells,65,Windsor,M,314,495
4,Barbara,Freeman,66,Windsor,F,747,Barbara,Freeman,73,Windsor,F,673,493
5,Barbara,Freeman,54,Windsor,F,912,Barbara,Freeman,73,Windsor,F,673,481
6,Barbara,Freeman,54,Windsor,F,912,Barbara,Freeman,66,Windsor,F,747,488
7,JOHN,BARNETT,52,WINN,M,942,JOHN,BARNETT,65,WINN,M,722,487
8,Jane,Davidson,63,Windsor,F,967,Jane,Davidson,54,Windsor,F,907,491
9,H,Meads,53,St. Andrew's Ward,F,990,H,Meads,53,St. Andrew's Ward,F,989,500


In [207]:
matched_pair.to_csv("Yiwei_match_first_3k_20191227.csv")